In [6]:
# 多类逻辑回归-从0开始
from mxnet import gluon
from mxnet import ndarray as nd

def transform(data, label):
    return data.astype('float32')/255, label.astype('float32')
mnist_train = gluon.data.vision.MNIST(train=True, transform=transform)
mnist_test = gluon.data.vision.MNIST(train=False, transform=transform)

# 提示AttributeError: 'module' object has no attribute 'FashionMNIST'
# mnist_train = gluon.data.vision.FashionMNIST(train=True, transform=transform)
# mnist_test = gluon.data.vision.FashionMNIST(train=False, transform=transform)

In [7]:
data, label = mnist_train[0]
('example shape: ', data.shape, 'label: ', label)
print len(mnist_train), len(mnist_test)

60000 10000


In [9]:
import matplotlib.pyplot as plt
def show_images(images):
    n = images.shape[0]
    _, figs = plt.subplots(1, n, figsize=(15, 15))
    for i in range(n):
        figs[i].imshow(images[i].reshape((28, 28)).asnumpy())
        figs[i].axes.get_xaxis().set_visible(False)
        figs[i].axes.get_yaxis().set_visible(False)
    plt.show()

def get_text_labels(label):
    text_labels = [
        't-shirt', 'trouser', 'pullover', 'dress,', 'coat',
        'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot'
    ]
    return [text_labels[int(i)] for i in label]

data, label = mnist_train[0:9]
show_images(data)
print(get_text_labels(label))

AttributeError: 'list' object has no attribute 'astype'

In [4]:
!pip show mxnet

Name: mxnet
Version: 0.11.0
Summary: MXNet is an ultra-scalable deep learning framework. This version uses openblas.
Home-page: https://github.com/dmlc/mxnet
Author: UNKNOWN
Author-email: UNKNOWN
License: Apache 2.0
Location: /usr/local/lib/python2.7/site-packages
Requires: numpy, graphviz


In [10]:
# 数据读取
batch_size = 256
train_data = gluon.data.DataLoader(mnist_train, batch_size, shuffle=True)
test_data = gluon.data.DataLoader(mnist_test, batch_size, shuffle=False)

In [11]:
# 初始化模型参数
num_inputs = 784
num_outputs = 10
W = nd.random_normal(shape=(num_inputs, num_outputs))
b = nd.random_normal(shape=num_outputs)
params = [W, b]

In [12]:
for param in params:
    param.attach_grad()

In [18]:
# 定义模型
from mxnet import nd
def softmax(X):
    exp = nd.exp(X)
    print X, exp
    partition = exp.sum(axis=1, keepdims=True)
    return exp / partition

In [19]:
X = nd.random_normal(shape=(2,5))
print X
X_prob = softmax(X)
print(X_prob)
print(X_prob.sum(axis=1))


[[ 0.51726657  0.23992962  0.63084751 -1.67366099 -2.12483311]
 [-0.78165877  1.41232193 -0.14712086  0.02452488  0.06619457]]
<NDArray 2x5 @cpu(0)>

[[ 0.51726657  0.23992962  0.63084751 -1.67366099 -2.12483311]
 [-0.78165877  1.41232193 -0.14712086  0.02452488  0.06619457]]
<NDArray 2x5 @cpu(0)> 
[[ 1.67743623  1.27115965  1.8792026   0.18755916  0.1194529 ]
 [ 0.45764625  4.10547686  0.86318964  1.02482808  1.0684346 ]]
<NDArray 2x5 @cpu(0)>


AttributeError: 'NDArray' object has no attribute 'sum'

In [20]:
def net(X):
    return softmax(nd.dot(X.reshape((-1, num_inputs)), W) + b)

In [21]:
# 交叉熵损失函数
def cross_entropy(yhat, y):
    return -nd.pick(nd.log(yhat), y)

In [22]:
# 计算精度
def accuracy(output, label):
    return nd.mean(output.argmax(axis=1)==label).asscaler()

In [23]:
def evaluate_accuracy(data_iterator, net):
    acc = 0
    for data, label in data_iterator:
        output = net(data)
        acc += accuracy(output, lable)
    return acc / len(data_iterator)

In [24]:
evaluate_accuracy(test_data, net)


[[  1.99287832  -2.42346668  -7.19146347 ...,  -0.50719297 -11.98436546
    9.8731823 ]
 [ -7.63568163   5.53134489 -11.26146507 ...,  -2.43261528 -12.98085976
   -3.25883746]
 [ -7.06860256   6.96442413   0.4660145  ...,  -1.76339948   1.51403904
   -7.50202179]
 ..., 
 [-14.48116207  -5.83258581   3.99703121 ...,   7.21555138   3.6995225
   -3.09768438]
 [  0.43560636   0.08347523   4.69026756 ...,   1.04162443  -1.22556019
   -3.3031652 ]
 [-10.04676628  -8.60235214  -1.0708679  ...,   1.29332483   4.0989418
   -0.48632687]]
<NDArray 256x10 @cpu(0)> 
[[  7.33662033e+00   8.86138901e-02   7.52986351e-04 ...,   6.02183580e-01
    6.24102904e-06   1.94029863e+04]
 [  4.82909323e-04   2.52483246e+02   1.28590200e-05 ...,   8.78068954e-02
    2.30400929e-06   3.84330526e-02]
 [  8.51422083e-04   1.05830530e+03   1.59363008e+00 ...,   1.71460986e-01
    4.54505157e+00   5.51967300e-04]
 ..., 
 [  5.13938573e-07   2.93048937e-03   5.44362984e+01 ...,   1.36042358e+03
    4.04279938e+01   

AttributeError: 'NDArray' object has no attribute 'sum'

In [25]:
# 训练
import sys
sys.path.append('..')
from utils import SGD
from mxnet import autograd

learning_rate = .1

for epoch in range(5):
    train_loss = 0.
    train_acc = 0.
    for data, label in train_data:
        with autograd.record():
            output = net(data)
            loss = cross_entropy(output, label)
        loss.backward()
        # 将梯度做平均，这样学习率会对batch size不那么敏感
        SGD(params, learning_rate/batch_size)

        train_loss += nd.mean(loss).asscalar()
        train_acc += accuracy(output, label)

    test_acc = evaluate_accuracy(test_data, net)
    print("Epoch %d. Loss: %f, Train acc %f, Test acc %f" % (
        epoch, train_loss/len(train_data), train_acc/len(train_data), test_acc))

ImportError: No module named utils

In [26]:
# 预测
data, label = mnist_test[0:9]
show_images(data)
print('true labels')
print(get_text_labels(label))

predicted_labels = net(data).argmax(axis=1)
print('predicted labels')
print(get_text_labels(predicted_labels.asnumpy()))

AttributeError: 'list' object has no attribute 'astype'